# SQL Operations Demo

One of the benefits of Spark is its support for SQL operations when it comes to streaming data. 

To do this, one first creates a `SparkSession` using the `SparkContext` that the `StreamingContext` is using. 


### Demo

For this demonstration, we will be using Spark and SQL to analyze data from a table of data on Wikipedia Articles.

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import *
from pyspark.sql import types
import sys


sc = SparkContext("local[2]", appName="PythonSqlNetworkWordCount")
sqlContext = SQLContext(sc)
lines = sc.textFile(sys.argv[1])
lines = lines.map(lambda x: str(x.encode('utf-8')))
lines = lines.map(lambda x: x.split(' ', 1))

fields = [types.StructField("topic", types.StringType(), True), types.StructField("title", types.StringType(), True)]
schema = types.StructType(fields)
schemaTopics = sqlContext.createDataFrame(lines, schema)
schemaTopics.registerTempTable("clusters")

def giveSparkSQLquery(input_topic):
    input_topic = input_topic.collect()
    if len(input_topic) > 0:
        input_topic = input_topic[0]
        query = "SELECT title FROM clusters WHERE topic = '" + input_topic + "'"
        results = sqlContext.sql(query)
        titles = results.map(lambda p: p.title)
        for title in titles.collect():
            print title

ssc = StreamingContext(sc, 3)
input_topic = ssc.socketTextStream("localhost", int(sys.argv[2]))
input_topic.foreachRDD(giveSparkSQLquery)
ssc.start()
ssc.awaitTermination()

    

## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#dataframe-and-sql-operations
2. https://spark.apache.org/docs/latest/sql-programming-guide.html